In [234]:
import numpy as np
from enum import IntEnum
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
#plt.style.use('seaborn-notebook')
sns.set_theme(context='notebook')
sns.set_style("whitegrid")
import matplotlib.colors as mcolors

In [235]:
class Action(IntEnum):
    up = 0
    right = 1
    down = 2
    left = 3

action_to_str = {
    Action.up : "up",
    Action.right : "right",
    Action.down : "down",
    Action.left : "left",
}

action_to_offset = {
    Action.up : (-1, 0),
    Action.right : (0, 1),
    Action.down : (1, 0),
    Action.left : (0, -1),
}

In [236]:
class GridWorld:

    def __init__(self, height, width, goal, goal_value=5.0, danger=[], danger_value=-5.0, blocked=[], noise=0.0):
        """
        Initialize the GridWorld environment.
        Creates a gridworld like MDP
         - height (int): Number of rows
         - width (int): Number of columns
         - goal (int): Index number of goal cell
         - goal_value (float): Reward given for goal cell
         - danger (list of int): Indices of cells marked as danger
         - danger_value (float): Reward given for danger cell
         - blocked (list of int): Indices of cells marked as blocked (can't enter)
         - noise (float): probability of resulting state not being what was expected
        """

        self._width = width
        self._height = height
        self._grid_values = [0 for _ in range(height * width)] # Initialize state values.
        self._goal_value = goal_value
        self._danger_value = danger_value
        self._goal_cell = goal
        self._danger_cells = danger
        self._blocked_cells = blocked
        self._noise = noise # Noise level in the environment.
        assert noise >= 0 and noise < 1 # Ensure valid noise value.
        self.create_next_values() # Initialize the next state values.


    def reset(self):
        """
        Reset the state values to their initial state.
        """
        self._grid_values = [0 for _ in range(self._height * self._width)]
        self.create_next_values()


    def _inbounds(self, state):
        """
        Check if a state index is within the grid boundaries.
        """
        return state >= 0 and state < self._width * self._height

    def _inbounds_rc(self, state_r, state_c):
        """
        Check if row and column indices are within the grid boundaries.
        """
        return state_r >= 0 and state_r < self._height and state_c >= 0 and state_c < self._width

    def _state_to_rc(self, state):
        """
        Convert a state index to row and column indices.
        """
        return state // self._width, state % self._width

    def _state_from_action(self, state, action):
        """
        Gets the state as a result of applying the given action
        """
        row, col = self._state_to_rc(state)
        dir_row, dir_col = action_to_offset[action] # returns the directional offset

        new_row = row + dir_row
        new_col = col + dir_col

        if not self._inbounds_rc(new_row, new_col):
            return state  # bump into outer edge > stay in place
        
        if (new_row, new_col) in {self._state_to_rc(b) for b in self._blocked_cells}:
            return state # return current state if walking into blocked

        return new_row * self._width + new_col # convert back into original state form

    def is_terminal(self, state):
        """
        Returns true if a state is terminal (goal, or danger)
        """
        return state in self._danger_cells or state == self._goal_cell

    def get_states(self):
        """
        Gets all non-terminal states in the environment
        """
        return [x for x in range(self._width*self._height) if x not in self._danger_cells and x != self._goal_cell]


    def get_actions(self, state):
        """
        Returns a list of valid actions given the current state
        """
        #Start with a full list, remove actions that lead to illegal states
        actions = [Action.up, Action.right, Action.down, Action.left]

        s_row, s_col = self._state_to_rc(state) # cconvert state to row,col format
        blocked = [self._state_to_rc(x) for x in self._blocked_cells]
        

        # check bounds and illegal states
        if (s_row - 1 < 0 or ((s_row-1, s_col) in blocked)): # Up
            actions.remove(Action.up)

        if (s_col + 1 >= self._width or ((s_row, s_col+1) in blocked)): # Right
            actions.remove(Action.right)

        if (s_row + 1 >= self._height or ((s_row+1, s_col) in blocked)): # Down
            actions.remove(Action.down)

        if (s_col - 1 < 0 or ((s_row, s_col-1) in blocked)): # Left
            actions.remove(Action.left)

        return actions



    def get_reward(self, state):
        """
        Get the reward for being in the current state
        """
        assert self._inbounds(state)
        # Reward is non-zero for danger or goal
        #TO DO:
        if state == self._goal_cell:
            return self._goal_value
        elif state in self._danger_cells:
            return self._danger_value
        
        return 0


    def get_transitions(self, state, action):
        """
        Get a list of transitions as a result of attempting the action in the current state
        Each item in the list is a dictionary, containing the probability of reaching that state and the state itself
        """
        #Assume action input is Action.___
        transitions = dict() # start with an empty dict
        possible_actions = [Action.up, Action.right, Action.down, Action.left]

        # Add the main state/action pair
        main_state = self._state_from_action(state, action)
        transitions[main_state] = 1 - self._noise

        #distribution noise
        noise_prob = self._noise / 3
        for a in possible_actions: # iterate threough all actions

            if a == action: #if we inserted this action alr.
                continue

            next_state = self._state_from_action(state, a)

            if next_state in transitions:
                transitions[next_state] += noise_prob
            else:
                transitions[next_state] = noise_prob
            

        return transitions


    def get_value(self, state):
        """
        Get the current value of the state
        """
        assert self._inbounds(state)
        return self._grid_values[state]

    def create_next_values(self):
        """
        Creates a temporary storage for state value updating
        If this is not used, then asynchronous updating may result in unexpected results
        To use properly, run this at the start of each iteration
        """
        self._new_grid_values = deepcopy(self._grid_values) #creates a temp storage value assigned to the GridWorld class

    def set_next_values(self):
        """
        Set the state values from the temporary copied values
        To use properly, run this at the end of each iteration
        """
        # imagine like a full commit of these new values
        self._grid_values = self._new_grid_values
        

    def set_value(self, state, value):
        """
        Set the value of the state into the temporary copy
        This value will not update into main storage until self.set_next_values() is called.
        """
        assert self._inbounds(state) #Ensure we are in bounds and pass in the assigned value
        self._new_grid_values[state] = value

    def solve_linear_system(self, discount_factor=1.0):
        """
        Solve the gridworld using a system of linear equations.
        :param discount_factor: The discount factor for future rewards.
        """

        states = list(range(self._width*self._height))

        #define an empty A and b to be solved
        A = np.zeros((len(states), len(states)))
        b = np.zeros(len(states))

        index = {s: i for i, s in enumerate(states)} # map states to equation indicies

        for st in states: # we iterate for every state to form this matrix
            i = index[st]
            
            #blocked states (value is 0)
            if st in self._blocked_cells:
                A[i,i] = 1
                b[i] = 0
                continue

            # terminal states
            if self.is_terminal(st):
                A[i,i] = 1
                b[i] = self.get_reward(st)
                continue

            # iterate through all possible actions in a NORMAL state
            A[i,i] = 1
            b[i] = self.get_reward(st)
            pi = 1.0 / len(self.get_actions(st))

            for a in self.get_actions(st):
                t = self.get_transitions(st,a) # gets the transitions of an action

                #iterate through each transition
                for next_st, p in t.items():
                    j = index[next_st]
                    A[i,j] -= discount_factor * pi * p

        # solve the matrix
        V = np.linalg.solve(A,b)

        for s, i in index.items():
            self._grid_values[s] = V[i]

        return V
    
    def __str__(self):
        """
        Pretty print the state values
        """
        out_str = ""
        for r in range(self._height):
            for c in range(self._width):
                cell = r * self._width + c
                if cell in self._blocked_cells:
                    out_str += "{:>6}".format("----")
                elif cell == self._goal_cell:
                    out_str += "{:>6}".format("GOAL")
                elif cell in self._danger_cells:
                    out_str += "{:>6.2f}".format(self._danger_value)
                else:
                    out_str += "{:>6.2f}".format(self._grid_values[cell])
                out_str += " "
            out_str += "\n"
        return out_str

In [237]:
# Initialize your GridWorld
simple_gw = GridWorld(height=5, width=5, goal=14, danger=[2, 18, 21], blocked=[6, 7, 11, 12], noise=0.0)

# Solve the linear system
values_grid = simple_gw.solve_linear_system(discount_factor=0.95)
print(simple_gw)
print("------")
print(values_grid)

 -3.36  -3.97  -5.00  -1.58   0.00 
 -3.10   ----   ----   0.00   1.58 
 -3.16   ----   ----   0.00   GOAL 
 -3.56  -4.01  -4.11  -5.00  -0.61 
 -4.07  -5.00  -3.98  -3.46  -1.93 

------
[-3.35549069e+00 -3.96885808e+00 -5.00000000e+00 -1.58333333e+00
  0.00000000e+00 -3.09533285e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.58333333e+00 -3.16099951e+00  0.00000000e+00
  0.00000000e+00  2.58375373e-16  5.00000000e+00 -3.55940297e+00
 -4.01350399e+00 -4.11482015e+00 -5.00000000e+00 -6.11823407e-01
 -4.06571641e+00 -5.00000000e+00 -3.98066491e+00 -3.45570063e+00
 -1.93207392e+00]


In [238]:
def value_iteration(gw, discount, tolerance=0.1):
    
    states = list(range(gw._width * gw._height))

    for it in range(1000):
        gw.create_next_values()
        d = 0.0 # delta

        for s in states:

            if s in gw._blocked_cells: # blocked
                new_v = 0.0

            elif gw.is_terminal(s): #terminal
                new_v = gw.get_reward(s)

            else: # other states
                r = gw.get_reward(s)
                best_q = -np.inf # koth style checking

                for a in gw.get_actions(s):
                    exp_v = 0.0
                    for s2, p in gw.get_transitions(s,a).items():
                        exp_v += p*gw.get_value(s2) # calculate the expected value given the probability and the next state for all actions and state transitions
                    q = r + discount * exp_v
                    if q > best_q:
                        best_q = q # the highest expected q is assigned to a state

                
                new_v = best_q
            gw.set_value(s, new_v) # assign our winning value to the state

            d = max(d, abs(new_v - gw.get_value(s)))

        gw.set_next_values()

        # stopping condition

        if d < tolerance:
            break
    
    return np.array([gw.get_value(s) for s in states])

        



In [239]:
# Initialize your GridWorld
simple_gw = GridWorld(height=5, width=5, goal=14, danger=[2, 18, 21], blocked=[6, 7, 11, 12], noise=0.0)
noisy_gw = GridWorld(height=5, width=5, goal=14, danger=[2, 18, 21], blocked=[6, 7, 11, 12], noise=0.2)
discount = 0.95
tolerance = 0.1

In [240]:
value_iteration(simple_gw, discount, 0.1)
print(simple_gw)

  2.99   2.84  -5.00   4.29   4.51 
  3.15   ----   ----   4.51   4.75 
  3.32   ----   ----   4.75   GOAL 
  3.49   3.68   3.87  -5.00   4.75 
  3.32  -5.00   4.07   4.29   4.51 



In [241]:
value_iteration(noisy_gw, discount, 0.1)
print(noisy_gw)

  0.33  -0.18  -5.00   3.42   4.28 
  0.48   ----   ----   4.26   4.63 
  0.59   ----   ----   4.01   GOAL 
  0.67   0.78   1.33  -5.00   3.97 
  0.20  -5.00   1.99   2.76   3.65 



In [ ]:
def policy_iteration(gw, discount, tolerance=0.1):
    """
    Input: 
    Output:

    evaluate, then iterate upon the policy
    """
    pass